In [1]:
from clickzetta.zettapark.session import Session
from clickzetta.zettapark.functions import expr

In [2]:
import json
# 从配置文件中读取参数
with open('security/config-uat.json', 'r') as config_file:
    config = json.load(config_file)

# 创建会话
session = Session.builder.configs(config).create()

In [3]:
#Concatenate columns
data=[("James","Bond"),("Scott","Varsa")] 
df=session.createDataFrame(data).toDF("col1","col2") 
df.withColumn("Name",expr(" col1 ||','|| col2")).show()

-------------------------------
|col1   |col2   |name         |
-------------------------------
|James  |Bond   |James,Bond   |
|Scott  |Varsa  |Scott,Varsa  |
-------------------------------



In [4]:
#Using CASE WHEN sql expression
data = [("James","M"),("Michael","F"),("Jen","")]
columns = ["name","gender"]
df = session.createDataFrame(data = data, schema = columns)
df2 = df.withColumn("gender", expr("CASE WHEN gender = 'M' THEN 'Male' " +
           "WHEN gender = 'F' THEN 'Female' ELSE 'unknown' END"))
df2.show()

---------------------
|name     |gender   |
---------------------
|James    |Male     |
|Michael  |Female   |
|Jen      |unknown  |
---------------------



In [5]:
#Add months from a value of another column
data=[("2019-01-23",1),("2019-06-24",2),("2019-09-20",3)] 
df=session.createDataFrame(data).toDF("date","increment") 
df.select(df.date,df.increment,
     expr("add_months(date,increment)")
  .alias("inc_date")).show()

# Providing alias using 'as'
df.select(df.date,df.increment,
     expr("""add_months(date,increment) as inc_date""")
  ).show()

---------------------------------------
|date        |increment  |inc_date    |
---------------------------------------
|2019-01-23  |1          |2019-02-23  |
|2019-06-24  |2          |2019-08-24  |
|2019-09-20  |3          |2019-12-20  |
---------------------------------------

---------------------------------------
|date        |increment  |inc_date    |
---------------------------------------
|2019-01-23  |1          |2019-02-23  |
|2019-06-24  |2          |2019-08-24  |
|2019-09-20  |3          |2019-12-20  |
---------------------------------------



In [6]:
# Add
df.select(df.date,df.increment,
     expr("increment + 5 as new_increment")
  ).show()

df.select("increment",expr("cast(increment as string) as str_increment")) \
  .printSchema()
#Use expr()  to filter the rows
data=[(100,2),(200,3000),(500,500)] 
df=session.createDataFrame(data).toDF("col1","col2") 
df.filter(expr("col1 == col2")).show()

------------------------------------------
|date        |increment  |new_increment  |
------------------------------------------
|2019-01-23  |1          |6              |
|2019-06-24  |2          |7              |
|2019-09-20  |3          |8              |
------------------------------------------

root
 |-- `increment`: IntegerType() (nullable = False)
 |-- `str_increment`: StringType() (nullable = False)
---------------
|col1  |col2  |
---------------
|500   |500   |
---------------



In [7]:
session.close()